In [1]:
import numpy as np
np.seterr(divide="ignore")

def switch_array_columns(array:np.ndarray, i1: int, i2: int) -> np.ndarray:
    new_array = array.copy()
    match new_array.shape:
        case (_,):
            new_array[[i1, i2]] = new_array[[i2, i1]]

        case (_, _):
            new_array[:,[i1, i2]] = new_array[:, [i2, i1]]

        case _:
            raise(ValueError)
            
    return new_array

def switch_array_rows(array:np.ndarray, i1: int, i2: int) -> np.ndarray:
    new_array = array.copy()
    new_array[[i1, i2]] = new_array[[i2, i1]]
            
    return new_array

def iteracion_simplex(c: np.ndarray, A: np.ndarray, b: np.ndarray, order = None) -> dict:
    if len(c.shape) == 1:
        raise ValueError("C debe ser un vector columna")
    if len(b.shape) == 1:
        raise ValueError("b debe ser un vector columna")
    
    m, n = A.shape

    if order is None:
        new_order = np.arange(0, n)
    else:
        new_order = order
    
    # Separar matriz A en variables básicas y no básicas
    B = A.copy()[:,0:m]
    N = A.copy()[:, m:n]
    
    #Costos c básicos y no básicos
    c_B = c.copy()[0:m, 0:]
    c_N = c.copy()[m:n, 0:]

    try:
        B_inv = np.linalg.inv(B)
    except Exception as _:
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_array_columns(A, i1, i2)
        c_new = switch_array_rows(c, i1, i2)
        new_order = switch_array_columns(new_order, i1, i2)
        return iteracion_simplex(c_new, A_new, b, new_order)
    
    # solución inicial z0 = z(x0)
    x0 = B_inv@b
    z0 = (c_B.T)@x0

    if any(x < 0 for x in x0):
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_array_columns(A, i1, i2)
        c_new = switch_array_rows(c, i1, i2)
        new_order = switch_array_columns(new_order, i1, i2)
        return iteracion_simplex(c_new, A_new, b, new_order)

    pi = c_B.T@B_inv
    # costos_reducidos = pi@N - c_N.T
    costos_reducidos = np.subtract(pi@N,c_N.T)

    b_barra = B_inv@b
    Y = B_inv@N

    result_x = np.zeros((n,1))
    result_x[0:m, 0:] = x0

    c_new = c.copy()
    res =  {
        "z": z0,
        "x": result_x,
        "pi": pi,
        "c_reducidos": costos_reducidos,
        "b_barra": b_barra,
        "Y": Y,
        "order": new_order,
        "pretty_print": {f"x{x}": result_x[idx, 0] for idx, x in np.ndenumerate(new_order)},
        "cnew": c_new,
        "check": {f"c{x}": c_new[idx, 0] for idx, x in np.ndenumerate(new_order)}
    }
    return res

In [2]:
def metodo_simplex_revisado(c: np.ndarray, A: np.ndarray, b: np.ndarray, order = None):
    m, n = A.shape
    inicial = iteracion_simplex(c, A, b, order)
    c_reducidos = inicial['c_reducidos']
    
    if all(x <= 0 for x in c_reducidos[0]):
        return inicial
    indice_no_basicas = np.argmax(c_reducidos)
    
    Y = inicial['Y']
    b_barra = inicial['b_barra']
    cocientes = np.divide(b_barra, np.vstack(Y[:, indice_no_basicas]))

    indice_basicas = np.where(cocientes > 0, cocientes, np.inf).argmin()

    i1, i2 = indice_basicas, indice_no_basicas + m

    A_new = switch_array_columns(A, i1, i2)
    c_new = switch_array_rows(c, i1, i2)
    order_new = switch_array_columns(inicial['order'], i1, i2)

    return metodo_simplex_revisado(c_new, A_new, b, order_new)

### Demo

In [3]:
A = np.array([
    [1,0,0,1,0],
    [0,1,0,0,1],
    [0,0,1,3,2]
])
c = np.vstack([0,0,0,-3,-5])
b = np.vstack([4, 6, 18])
order = np.array([2,3,4,0,1])
metodo_simplex_revisado(c,A,b,order)

{'z': array([[-36.]]),
 'x': array([[2.],
        [6.],
        [2.],
        [0.],
        [0.]]),
 'pi': array([[ 0., -3., -1.]]),
 'c_reducidos': array([[-1., -3.]]),
 'b_barra': array([[2.],
        [6.],
        [2.]]),
 'Y': array([[-0.33333333,  0.66666667],
        [ 0.        ,  1.        ],
        [ 0.33333333, -0.66666667]]),
 'order': array([2, 1, 0, 4, 3]),
 'pretty_print': {'x2': array([2.]),
  'x1': array([6.]),
  'x0': array([2.]),
  'x4': array([0.]),
  'x3': array([0.])},
 'cnew': array([[ 0],
        [-5],
        [-3],
        [ 0],
        [ 0]]),
 'check': {'c2': array([0]),
  'c1': array([-5]),
  'c0': array([-3]),
  'c4': array([0]),
  'c3': array([0])}}

Con otro orden se comporta raro

In [4]:
A = np.array([[3, 2, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]])
c = np.vstack([-3,-5,0,0,0])
b = np.vstack([18, 4, 6])
order = np.arange(0,5)
metodo_simplex_revisado(c,A,b,)


{'z': array([[-36.]]),
 'x': array([[2.],
        [6.],
        [2.],
        [0.],
        [0.]]),
 'pi': array([[-1.,  0., -3.]]),
 'c_reducidos': array([[-1., -3.]]),
 'b_barra': array([[2.],
        [6.],
        [2.]]),
 'Y': array([[ 0.33333333, -0.66666667],
        [ 0.        ,  1.        ],
        [-0.33333333,  0.66666667]]),
 'order': array([3, 1, 0, 2, 4]),
 'pretty_print': {'x3': array([2.]),
  'x1': array([6.]),
  'x0': array([2.]),
  'x2': array([0.]),
  'x4': array([0.])},
 'cnew': array([[-3],
        [-5],
        [ 0],
        [ 0],
        [ 0]]),
 'check': {'c3': array([-3]),
  'c1': array([-5]),
  'c0': array([0]),
  'c2': array([0]),
  'c4': array([0])}}

# Ejercicio de la Tarea

Demanda:
$$ P_1 + P_2 + P_3 = 475 $$

Límites superiores de operación:
$$ P_1 + s_1 = 175 $$
$$ P_2 + s_2 = 300 $$
$$ P_3 + s_3 = 150 $$

Límites inferiores:
$$ P_3 - s_4 =50 $$

Restricción de las lineas:
$$ P_1 + s_5 = 160 $$
$$-P_3 + s_6 = -25 $$

$$\begin{bmatrix}
    1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
    1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\ 
    0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0 & 0 &-1 & 0 & 0 \\
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
    0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 &-1 
\end{bmatrix}
 = \begin{bmatrix}
    475 \\ 175 \\ 300 \\ 150 \\ 50 \\ 160 \\ 25
\end{bmatrix}
$$

Reducida:
$$\begin{bmatrix}
    1 & 1 & 1 & 0 & 0 & 0 & 0 \\
    1 & 0 & 0 & 1 & 0 & 0 & 0 \\ 
    0 & 1 & 0 & 0 & 1 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0 & 1 & 0 \\
    0 & 0 & 1 & 0 & 0 & 0 &-1 \\

\end{bmatrix}
 = \begin{bmatrix}
    475 \\ 160 \\ 300 \\ 150 \\ 50
\end{bmatrix}
$$

In [5]:
from scipy.optimize import linprog

c = np.array([50,60,75])
A_eq = [[1, 1, 1]]
b_eq = [475]

bounds = [(0,160), (0,300), (50,150)]
res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 27650.0
              x: [ 1.600e+02  2.650e+02  5.000e+01]
            nit: 0
          lower:  residual: [ 1.600e+02  2.650e+02  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  1.500e+01]
          upper:  residual: [ 0.000e+00  3.500e+01  1.000e+02]
                 marginals: [-1.000e+01  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [ 6.000e+01]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [6]:
c = np.vstack([50, 60, 75, 0, 0, 0, 0, 0, 0])
A = np.array([
    [1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0,-1, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, -1],
])
b = np.vstack([475, 175, 300, 150, 50, 160, 25])

metodo_simplex_revisado(c, A, b)

{'z': array([[27650.]]),
 'x': array([[ 15.],
        [100.],
        [ 25.],
        [265.],
        [ 35.],
        [160.],
        [ 50.],
        [  0.],
        [  0.]]),
 'pi': array([[ 60.,   0.,   0.,   0.,  15., -10.,   0.]]),
 'c_reducidos': array([[-10., -15.]]),
 'b_barra': array([[ 15.],
        [100.],
        [ 25.],
        [265.],
        [ 35.],
        [160.],
        [ 50.]]),
 'Y': array([[-1.,  0.],
        [ 0.,  1.],
        [ 0., -1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1.,  0.],
        [ 0., -1.]]),
 'order': array([3, 5, 8, 1, 4, 0, 2, 7, 6]),
 'pretty_print': {'x3': array([15.]),
  'x5': array([100.]),
  'x8': array([25.]),
  'x1': array([265.]),
  'x4': array([35.]),
  'x0': array([160.]),
  'x2': array([50.]),
  'x7': array([0.]),
  'x6': array([0.])},
 'cnew': array([[ 0],
        [ 0],
        [ 0],
        [60],
        [ 0],
        [50],
        [75],
        [ 0],
        [ 0]]),
 'check': {'c3': array([0]),
  'c5': array([0]),
  'c8'

In [7]:
def build_standard_matrices(c: np.ndarray, A_eq: np.ndarray, b_eq:np.ndarray, bounds=list[tuple]) -> dict:
    eq_rows, eq_colums = A_eq.shape
    lower_bounds, upper_bounds = zip(*bounds)
    lower_bounds = list(filter(lambda x: x > 0, lower_bounds))
    upper_bounds = list(filter(lambda x: x > 0, upper_bounds))
    
    upper_count = len(upper_bounds)
    lower_count = len(lower_bounds)
    rows = eq_rows + upper_count + lower_count
    cols = eq_colums + upper_count + lower_count

    slack_variable_count = rows - eq_rows
    normal_variable_count = eq_colums

    A = np.zeros(shape=(rows, cols))
    #equalities
    A[0:eq_rows, cols-eq_colums:cols] = A_eq
    #Fill in basic variables last
    A[eq_rows:eq_rows+eq_colums, cols-eq_colums:cols] = np.eye(normal_variable_count)
    # then slack upper limits
    A[eq_rows:eq_rows+eq_colums, 0:eq_colums] = np.eye(normal_variable_count)
    
    lower_bound_count = 0 
    for idx, (low, _) in enumerate(bounds):
        if low > 0:
            A[rows-1-lower_bound_count, cols - (normal_variable_count - idx)] = 1
            A[rows-1-lower_bound_count, slack_variable_count-lower_bound_count-1] = -1
            lower_bound_count += 1

    c_new = np.zeros(shape=(cols,1))
    c_new[cols-eq_colums: cols, 0] = c

    b_new = np.zeros(shape=(rows, 1))
    b_new[0:eq_rows] = b_eq
    b_new[eq_rows:eq_rows+upper_count] = np.vstack(upper_bounds)
    b_new[rows-lower_bound_count:rows] = np.vstack([x for x in reversed(lower_bounds)])
    return {
        'c': c_new,
        'A': A,
        'b': b_new,
        'basic_vars': normal_variable_count,
        'slack_vars': slack_variable_count
    }
    

In [8]:
def metodo_simplex_talegon(c: np.ndarray, A_eq: np.ndarray, b_eq:np.ndarray, bounds=list[tuple]) -> dict:
    standard_form = build_standard_matrices(c, A_eq, b_eq, bounds)
    c_std = standard_form['c']
    A_std = standard_form['A']
    b_std = standard_form['b']

    basic_vars = standard_form['basic_vars']
    slack_vars = standard_form['slack_vars']

    order = [x for x in range(basic_vars, slack_vars+basic_vars)]
    order += [x for x in range(0, basic_vars)]
    order = np.array(order)


    # print(A_std, b_std, "\n")

    result = metodo_simplex_revisado(c_std, A_std, b_std, order)
    # result = metodo_simplex_revisado(c_std, A_std, b_std)

    return result

In [9]:
c = np.array([50,60,75])
A_eq = np.array([[1, 1, 1]])
b_eq = np.array(475)

bounds = [(0,160), (0,300), (50,150)]
metodo_simplex_talegon(c, A_eq, b_eq, bounds)

{'z': array([[27650.]]),
 'x': array([[265.],
        [ 50.],
        [ 35.],
        [100.],
        [160.],
        [  0.],
        [  0.]]),
 'pi': array([[ 60., -10.,   0.,   0.,  15.]]),
 'c_reducidos': array([[-15., -10.]]),
 'b_barra': array([[265.],
        [ 50.],
        [ 35.],
        [100.],
        [160.]]),
 'Y': array([[ 1., -1.],
        [-1.,  0.],
        [-1.,  1.],
        [ 1.,  0.],
        [ 0.,  1.]]),
 'order': array([1, 2, 4, 5, 0, 6, 3]),
 'pretty_print': {'x1': array([265.]),
  'x2': array([50.]),
  'x4': array([35.]),
  'x5': array([100.]),
  'x0': array([160.]),
  'x6': array([0.]),
  'x3': array([0.])},
 'cnew': array([[60.],
        [75.],
        [ 0.],
        [ 0.],
        [50.],
        [ 0.],
        [ 0.]]),
 'check': {'c1': array([60.]),
  'c2': array([75.]),
  'c4': array([0.]),
  'c5': array([0.]),
  'c0': array([50.]),
  'c6': array([0.]),
  'c3': array([0.])}}